##Notebook Generated by ChimeraCat


In [ ]:
# Generated by ChimeraCat
    #  /\___/\  ChimeraCat
    # ( o   o )  Modular Python Fusion
    # (  =^=  ) 
    #  (______)  Generated: 2024-11-20 04:29:43
    #
    # Compression Level: full
    
# External imports
import abc
import dataclasses
import datetime
import enum
import librosa
import numpy as np
import pathlib
import soundfile as sf
import spleeter.separator
import tensorflow as tf
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data
import typing

# Combined module code


# From analysis\artifacts\base.py
from abc import ABC, abstractmethod
import numpy as np
import torch
import torch.nn as nn
from dataclasses import dataclass
from typing import Dict, List, Optional, Tuple
from pathlib import Path

@dataclass
class ValidationMetrics:
    """Stores comparison metrics between original and processed audio"""
    phase_coherence: float
    frequency_response: Dict[str, float]
    signal_to_noise: float
    processing_time: float
    memory_usage: float
    
    def as_dict(self) -> Dict[str, float]:
        return {
            "phase_coherence": self.phase_coherence,
            "freq_response_mean": np.mean(list(self.frequency_response.values())),
            "freq_response_std": np.std(list(self.frequency_response.values())),
            "snr": self.signal_to_noise,
            "processing_time": self.processing_time,
            "memory_usage": self.memory_usage
        }

class ArtifactProcessor(ABC):
    """Base class for different artifact processing approaches"""
    
    def __init__(self, sample_rate: int = 44100):
        self.sample_rate = sample_rate
        self.frequency_bands = {
            "sub_bass": (20, 60),
            "bass": (60, 250),
            "low_mid": (250, 500),
            "mid": (500, 2000),
            "high_mid": (2000, 4000),
            "presence": (4000, 6000),
            "brilliance": (6000, 20000)
        }
    
    @abstractmethod
    def process(self, 
                audio_segment: AudioSegment,
                artifact_type: str) -> AudioSegment:
        """Process audio to remove specified artifact type"""
        pass
    
    def validate(self, 
                clean: AudioSegment,
                processed: AudioSegment,
                original_artifacts: AudioSegment) -> ValidationMetrics:
        """Calculate validation metrics comparing processed output to clean reference"""
        
        # Calculate phase coherence
        phase_coherence = self._measure_phase_coherence(
            clean.audio, processed.audio
        )
        
        # Analyze frequency response
        freq_response = self._analyze_frequency_response(
            clean.audio, processed.audio
        )
        
        # Calculate SNR using original artifacts as noise reference
        snr = self._calculate_snr(
            clean.audio,
            processed.audio,
            original_artifacts.audio
        )
        
        return ValidationMetrics(
            phase_coherence=phase_coherence,
            frequency_response=freq_response,
            signal_to_noise=snr,
            processing_time=0.0,  # Set by wrapper
            memory_usage=0.0  # Set by wrapper
        )
    
    def _measure_phase_coherence(self,
                               clean: np.ndarray,
                               processed: np.ndarray) -> float:
        """Measure phase coherence between clean and processed audio"""
        # Use STFT for phase analysis
        n_fft = 2048
        hop_length = 512
        
        clean_stft = librosa.stft(clean, n_fft=n_fft, hop_length=hop_length)
        proc_stft = librosa.stft(processed, n_fft=n_fft, hop_length=hop_length)
        
        # Calculate phase difference
        clean_phase = np.angle(clean_stft)
        proc_phase = np.angle(proc_stft)
        phase_diff = np.abs(clean_phase - proc_phase)
        
        # Return mean phase coherence (1 = perfect, 0 = random)
        return float(np.mean(np.cos(phase_diff)))
    
    def _analyze_frequency_response(self,
                                  clean: np.ndarray,
                                  processed: np.ndarray) -> Dict[str, float]:
        """Analyze frequency response preservation in different bands"""
        results = {}
        
        for band_name, (low, high) in self.frequency_bands.items():
            # Filter both signals to band
            clean_band = self._bandpass_filter(clean, low, high)
            proc_band = self._bandpass_filter(processed, low, high)
            
            # Calculate RMS difference
            clean_rms = np.sqrt(np.mean(clean_band ** 2))
            proc_rms = np.sqrt(np.mean(proc_band ** 2))
            
            # Store ratio (1 = perfect preservation)
            results[band_name] = proc_rms / clean_rms if clean_rms > 0 else 0.0
            
        return results
    
    def _bandpass_filter(self,
                        audio: np.ndarray,
                        low_freq: float,
                        high_freq: float) -> np.ndarray:
        """Apply bandpass filter to audio"""
        nyquist = self.sample_rate // 2
        low = low_freq / nyquist
        high = high_freq / nyquist
        
        # Design filter
        b, a = scipy.signal.butter(4, [low, high], btype='band')
        
        # Apply filter
        return scipy.signal.filtfilt(b, a, audio)
    
    def _calculate_snr(self,
                      clean: np.ndarray,
                      processed: np.ndarray,
                      noise: np.ndarray) -> float:
        """Calculate signal-to-noise ratio"""
        signal_power = np.mean(clean ** 2)
        noise_power = np.mean((processed - clean) ** 2)
        
        return 10 * np.log10(signal_power / noise_power) if noise_power > 0 else float('inf')

class ControlNetProcessor(ArtifactProcessor):
    """ControlNet-based artifact processor"""
    
    def __init__(self, model_path: Optional[Path] = None):
        super().__init__()
        self.model = PhaseAwareControlNet(...)  # Initialize from earlier code
        if model_path:
            self.model.load_state_dict(torch.load(model_path))
    
    def process(self,
                audio_segment: AudioSegment,
                artifact_type: str) -> AudioSegment:
        # Convert to spectrogram
        spec = self._audio_to_spectrogram(audio_segment)
        
        # Process through ControlNet
        processed_spec = self.model(spec)
        
        # Convert back to audio
        return self._spectrogram_to_audio(processed_spec)

class SignalProcessor(ArtifactProcessor):
    """Direct signal-domain processor"""
    
    def __init__(self):
        super().__init__()
        
    def process(self,
                audio_segment: AudioSegment,
                artifact_type: str) -> AudioSegment:
        # Implement direct signal processing approach
        # (We'll flesh this out as we identify specific artifacts)
        pass

class HybridProcessor(ArtifactProcessor):
    """Hybrid approach combining latent and signal processing"""
    
    def __init__(self):
        super().__init__()
        self.controlnet = ControlNetProcessor()
        self.signal = SignalProcessor()
        
    def process(self,
                audio_segment: AudioSegment,
                artifact_type: str) -> AudioSegment:
        # Route to appropriate processor based on artifact type
        # Or combine both approaches
        pass

def run_validation(
    processor: ArtifactProcessor,
    test_cases: List[Tuple[AudioSegment, AudioSegment, AudioSegment]],
    artifact_types: List[str]
) -> Dict[str, List[ValidationMetrics]]:
    """Run validation suite on processor"""
    
    results = {artifact_type: [] for artifact_type in artifact_types}
    
    for clean, artifacts, mixed in test_cases:
        for artifact_type in artifact_types:
            # Time and memory measurement wrapper
            start_time = time.time()
            start_mem = psutil.Process().memory_info().rss
            
            # Process audio
            processed = processor.process(mixed, artifact_type)
            
            # Get metrics
            metrics = processor.validate(clean, processed, artifacts)
            metrics.processing_time = time.time() - start_time
            metrics.memory_usage = (
                psutil.Process().memory_info().rss - start_mem
            ) / 1024 / 1024  # MB
            
            results[artifact_type].append(metrics)
    
    return results

# From analysis\artifacts\high_freq.py
class HighFrequencyArtifactPreprocessor(nn.Module):
    def __init__(self, threshold_freq: float = 11000, sample_rate: int = 44100):
        super().__init__()
        self.threshold_freq = threshold_freq
        self.sample_rate = sample_rate
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: RGBA spectrogram tensor (B, 4, H, W)
        Returns:
            Single-channel attention map (B, 1, H, W)
        """
        # Extract magnitude from blue channel
        magnitude = x[:, 2:3]  # (B, 1, H, W)
        
        # Calculate frequency bins
        freq_bins = torch.linspace(0, self.sample_rate/2, magnitude.shape[2])
        
        # Create frequency mask
        mask = torch.ones_like(magnitude)
        high_freq_idx = (freq_bins > self.threshold_freq).nonzero()
        
        if high_freq_idx.numel() > 0:
            # Analyze high frequency content
            high_freq_content = magnitude[:, :, high_freq_idx.squeeze():]
            
            # Detect potential artifacts using local statistics
            mean = torch.mean(high_freq_content, dim=-1, keepdim=True)
            std = torch.std(high_freq_content, dim=-1, keepdim=True)
            
            # Create attention map
            attention = torch.sigmoid(
                (high_freq_content - mean) / (std + 1e-6)
            )
            
            # Insert back into full mask
            mask[:, :, high_freq_idx.squeeze():] = attention
        
        return mask

def generate_training_pair(
    clean_audio: torch.Tensor,
    separated_audio: torch.Tensor,
    preprocessor: HighFrequencyArtifactPreprocessor
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Generate training pair for ControlNet
    Returns:
        condition: Control signal from preprocessor
        input_spec: Separated audio spectrogram
        target_spec: Clean audio spectrogram
    """
    # Convert both to spectrograms
    clean_spec = audio_to_spectrogram(clean_audio)
    sep_spec = audio_to_spectrogram(separated_audio)
    
    # Generate control signal
    condition = preprocessor(sep_spec)
    
    return condition, sep_spec, clean_spec

# From analysis\artifacts\preprocessor.py
class HighFrequencyArtifactPreprocessor(nn.Module):
    def __init__(self, threshold_freq: float = 11000, sample_rate: int = 44100):
        super().__init__()
        self.threshold_freq = threshold_freq
        self.sample_rate = sample_rate
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: RGBA spectrogram tensor (B, 4, H, W)
        Returns:
            Single-channel attention map (B, 1, H, W)
        """
        # Extract magnitude from blue channel
        magnitude = x[:, 2:3]  # (B, 1, H, W)
        
        # Calculate frequency bins
        freq_bins = torch.linspace(0, self.sample_rate/2, magnitude.shape[2])
        
        # Create frequency mask
        mask = torch.ones_like(magnitude)
        high_freq_idx = (freq_bins > self.threshold_freq).nonzero()
        
        if high_freq_idx.numel() > 0:
            # Analyze high frequency content
            high_freq_content = magnitude[:, :, high_freq_idx.squeeze():]
            
            # Detect potential artifacts using local statistics
            mean = torch.mean(high_freq_content, dim=-1, keepdim=True)
            std = torch.std(high_freq_content, dim=-1, keepdim=True)
            
            # Create attention map
            attention = torch.sigmoid(
                (high_freq_content - mean) / (std + 1e-6)
            )
            
            # Insert back into full mask
            mask[:, :, high_freq_idx.squeeze():] = attention
        
        return mask

def generate_training_pair(
    clean_audio: torch.Tensor,
    separated_audio: torch.Tensor,
    preprocessor: HighFrequencyArtifactPreprocessor
) -> Tuple[torch.Tensor, torch.Tensor, torch.Tensor]:
    """
    Generate training pair for ControlNet
    Returns:
        condition: Control signal from preprocessor
        input_spec: Separated audio spectrogram
        target_spec: Clean audio spectrogram
    """
    # Convert both to spectrograms
    clean_spec = audio_to_spectrogram(clean_audio)
    sep_spec = audio_to_spectrogram(separated_audio)
    
    # Generate control signal
    condition = preprocessor(sep_spec)
    
    return condition, sep_spec, clean_spec

# From core\audio.py
from dataclasses import dataclass
import numpy as np
import librosa
from typing import Optional

@dataclass
class AudioSegment:
    """Data class for audio segments with their metadata"""
    audio: np.ndarray
    sample_rate: int = 44100
    start_time: float = 0.0
    duration: float = 0.0

    @property
    def is_stereo(self) -> bool:
        stereo = len(self.audio.shape) == 2 and (
            self.audio.shape[0] == 2 or self.audio.shape[1] == 2
        )
        print(f"is_stereo check - shape: {self.audio.shape}, result: {stereo}")
        return stereo

    @property
    def is_mono(self) -> bool:
        mono = len(self.audio.shape) == 1 or (
            len(self.audio.shape) == 2 and (
                self.audio.shape[0] == 1 or self.audio.shape[1] == 1
            )
        )
        print(f"is_mono check - shape: {self.audio.shape}, result: {mono}")
        return mono

    def to_mono(self) -> 'AudioSegment':
        """Convert to mono if stereo"""
        print(f"to_mono - input shape: {self.audio.shape}")
        
        if self.is_mono:
            print("Already mono, returning as is")
            return self
            
        # Handle different stereo formats
        if len(self.audio.shape) == 2:
            if self.audio.shape[0] == 2:
                # (2, samples) format
                mono_audio = librosa.to_mono(self.audio)
            elif self.audio.shape[1] == 2:
                # (samples, 2) format
                mono_audio = librosa.to_mono(self.audio.T)
            else:
                raise ValueError(f"Unexpected audio shape: {self.audio.shape}")
        else:
            raise ValueError(f"Cannot convert shape {self.audio.shape} to mono")
            
        print(f"to_mono - output shape: {mono_audio.shape}")
        
        return AudioSegment(
            audio=mono_audio,
            sample_rate=self.sample_rate,
            start_time=self.start_time,
            duration=self.duration
        )

    @property
    def duration_seconds(self) -> float:
        """Get duration in seconds based on audio shape and sample rate"""
        if self.is_stereo:
            return self.audio.shape[1] / self.sample_rate
        return len(self.audio) / self.sample_rate

# From core\config.py
from dataclasses import dataclass
from enum import Enum, auto
from pathlib import Path
from typing import Optional

class SeparatorBackend(Enum):
    SPLEETER = auto()
    DEMUCS = auto()
    MDX = auto()

@dataclass
class SeparationProfile:
    """Processing profile configuration"""
    backend: SeparatorBackend
    preserve_high_freq: bool = False
    target_sample_rate: int = 44100
    min_segment_length: float = 5.0
    
    # Enhancement settings
    use_phase_aware_controlnet: bool = False
    use_high_freq_processor: bool = True
    artifact_reduction_config: Optional[ProcessingConfig] = None
    controlnet_model_path: Optional[Path] = None

# From core\types.py
from dataclasses import dataclass
from pathlib import Path
from typing import Dict, Optional, Any
# from .audio import AudioSegment  # Original relative import

@dataclass
class ProcessingConfig:
    """Configuration for audio processing"""
    # Audio parameters
    sample_rate: int = 44100
    n_fft: int = 2048
    hop_length: int = 512
    pad_mode: str = 'constant'
    
    # Image processing
    image_scale_factor: float = 1.0
    image_chunk_size: int = 512
    
    # ControlNet parameters
    enable_controlnet: bool = True
    controlnet_strength: float = 0.75
    guidance_scale: float = 7.5
    num_inference_steps: int = 50
    
    # Optimization
    torch_dtype: str = 'float16'
    attention_slice_size: Optional[int] = 1
    enable_xformers: bool = True
    enable_cuda_graph: bool = False
    
    # Artifact processing
    artifact_threshold_freq: float = 11000
    artifact_smoothing_sigma: float = 1.0
    temporal_smoothing: float = 2.0

@dataclass
class SeparationResult:
    """Data class for storing separation results"""
    clean_vocal: AudioSegment
    separated_vocal: AudioSegment
    enhanced_vocal: Optional[AudioSegment]
    accompaniment: AudioSegment
    mixed: AudioSegment
    file_paths: Dict[str, Path]
    analysis_path: Optional[Path] = None
    phase_analysis: Optional[Dict[str, Any]] = None
    artifact_analysis: Optional[Dict[str, str]] = None

# From enhancement\controlnet.py
import torch
import torch.nn as nn
from typing import Optional, List, Tuple

class ArtifactDetector(nn.Module):
    """Preprocessor that generates artifact control maps"""
    def __init__(self):
        super().__init__()
        # Lightweight conv layers for artifact detection
        self.detector = nn.Sequential(
            nn.Conv2d(4, 16, 3, padding=1),  # 4 channels: RGB + phase
            nn.ReLU(),
            nn.Conv2d(16, 16, 3, padding=1),
            nn.ReLU(),
            nn.Conv2d(16, 1, 1)  # Single channel artifact map
        )
        
    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        Args:
            x: RGBA tensor where A channel contains phase information
        Returns:
            Single-channel artifact probability map
        """
        return torch.sigmoid(self.detector(x))

class PhaseAwareZeroConv(nn.Module):
    """Zero convolution block that preserves phase information"""
    def __init__(
        self, 
        input_channels: int,
        output_channels: int,
        phase_channels: int = 1
    ):
        super().__init__()
        self.main_conv = nn.Conv2d(input_channels, output_channels, 1)
        self.phase_conv = nn.Conv2d(phase_channels, phase_channels, 1)
        
        # Initialize to zero
        nn.init.zeros_(self.main_conv.weight)
        nn.init.zeros_(self.main_conv.bias)
        nn.init.zeros_(self.phase_conv.weight)
        nn.init.zeros_(self.phase_conv.bias)
        
    def forward(self, x: torch.Tensor, control: torch.Tensor) -> Tuple[torch.Tensor, torch.Tensor]:
        # Split main features and phase
        features, phase = x[:, :-1], x[:, -1:]
        
        # Apply zero convolutions
        out_features = self.main_conv(features * control)
        out_phase = self.phase_conv(phase * control)
        
        return torch.cat([out_features, out_phase], dim=1)

class PhaseAwareControlNet(nn.Module):
    """ControlNet adaptation for phase-aware spectrogram processing"""
    def __init__(
        self,
        base_model: nn.Module,
        control_channels: int = 1,
        phase_channels: int = 1
    ):
        super().__init__()
        self.base_model = base_model
        self.artifact_detector = ArtifactDetector()
        
        # Create zero conv layers for each injection point
        self.zero_convs = nn.ModuleList([
            PhaseAwareZeroConv(
                base_model.get_feature_channels(i),
                base_model.get_feature_channels(i),
                phase_channels
            )
            for i in range(base_model.num_injection_points)
        ])
        
    def forward(
        self, 
        x: torch.Tensor,
        timestep: Optional[torch.Tensor] = None,
        context: Optional[torch.Tensor] = None
    ) -> torch.Tensor:
        # Generate control signal
        control = self.artifact_detector(x)
        
        # Get intermediate features from frozen base model
        features = self.base_model.get_feature_pyramid(x, timestep, context)
        
        # Apply controlled features through zero convs
        controlled_features = []
        for feat, zero_conv in zip(features, self.zero_convs):
            controlled_features.append(zero_conv(feat, control))
            
        return self.base_model.forward_with_features(
            x,
            controlled_features,
            timestep,
            context
        )

# From enhancement\training.py
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F

class ArtifactDataset(Dataset):
    def __init__(self, 
                 clean_paths: List[str],
                 separated_paths: List[str],
                 preprocessor: HighFrequencyArtifactPreprocessor):
        self.clean_paths = clean_paths
        self.separated_paths = separated_paths
        self.preprocessor = preprocessor
        
    def __len__(self):
        return len(self.clean_paths)
        
    def __getitem__(self, idx):
        clean = load_audio(self.clean_paths[idx])
        separated = load_audio(self.separated_paths[idx])
        
        condition, input_spec, target_spec = generate_training_pair(
            clean, separated, self.preprocessor
        )
        
        return {
            'condition': condition,
            'input': input_spec,
            'target': target_spec
        }

class ControlNetTrainer:
    def __init__(self,
                 model: PhaseAwareControlNet,
                 preprocessor: HighFrequencyArtifactPreprocessor,
                 learning_rate: float = 1e-4,
                 device: str = 'cuda'):
        self.model = model.to(device)
        self.preprocessor = preprocessor.to(device)
        self.device = device
        
        # Freeze base model, train only ControlNet components
        for param in self.model.base_model.parameters():
            param.requires_grad = False
            
        self.optimizer = torch.optim.AdamW(
            filter(lambda p: p.requires_grad, self.model.parameters()),
            lr=learning_rate
        )
        
    def train_step(self, batch: Dict[str, torch.Tensor]) -> Dict[str, float]:
        self.optimizer.zero_grad()
        
        # Move data to device
        condition = batch['condition'].to(self.device)
        input_spec = batch['input'].to(self.device)
        target_spec = batch['target'].to(self.device)
        
        # Forward pass
        output = self.model(input_spec, control=condition)
        
        # Compute losses
        losses = {
            'magnitude': F.mse_loss(output[:, :-1], target_spec[:, :-1]),
            'phase': F.mse_loss(output[:, -1:], target_spec[:, -1:]),
            'frequency': self.frequency_loss(output, target_spec)
        }
        
        # Combined loss
        total_loss = sum(losses.values())
        total_loss.backward()
        
        # Gradient clipping
        torch.nn.utils.clip_grad_norm_(
            filter(lambda p: p.requires_grad, self.model.parameters()),
            max_norm=1.0
        )
        
        self.optimizer.step()
        
        return {k: v.item() for k, v in losses.items()}
    
    def frequency_loss(self, pred: torch.Tensor, target: torch.Tensor) -> torch.Tensor:
        """Frequency-domain loss focusing on high frequencies"""
        # Convert to frequency domain
        pred_fft = torch.fft.rfft2(pred[:, :-1])  # Exclude phase channel
        target_fft = torch.fft.rfft2(target[:, :-1])
        
        # Weight higher frequencies more
        freq_weights = torch.linspace(1.0, 2.0, pred_fft.size(-1))
        freq_weights = freq_weights.to(self.device)
        
        return F.mse_loss(
            pred_fft * freq_weights,
            target_fft * freq_weights
        )
    
    def train(self,
             train_loader: DataLoader,
             val_loader: Optional[DataLoader] = None,
             epochs: int = 100,
             save_dir: str = 'checkpoints'):
        best_val_loss = float('inf')
        
        for epoch in range(epochs):
            self.model.train()
            train_losses = []
            
            for batch in train_loader:
                losses = self.train_step(batch)
                train_losses.append(losses)
            
            # Validation
            if val_loader is not None:
                val_loss = self.validate(val_loader)
                
                if val_loss < best_val_loss:
                    best_val_loss = val_loss
                    self.save_checkpoint(f'{save_dir}/best.pt')
            
            if epoch % 10 == 0:
                self.save_checkpoint(f'{save_dir}/epoch_{epoch}.pt')
    
    def validate(self, val_loader: DataLoader) -> float:
        self.model.eval()
        val_losses = []
        
        with torch.no_grad():
            for batch in val_loader:
                condition = batch['condition'].to(self.device)
                input_spec = batch['input'].to(self.device)
                target_spec = batch['target'].to(self.device)
                
                output = self.model(input_spec, control=condition)
                loss = F.mse_loss(output, target_spec)
                val_losses.append(loss.item())
        
        return sum(val_losses) / len(val_losses)
    
    def save_checkpoint(self, path: str):
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict()
        }, path)
    
    def load_checkpoint(self, path: str):
        checkpoint = torch.load(path)
        self.model.load_state_dict(checkpoint['model_state_dict'])
        self.optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

def prepare_training(
    clean_dir: str,
    separated_dir: str,
    batch_size: int = 8,
    val_split: float = 0.1
) -> Tuple[DataLoader, DataLoader]:
    # Get audio paths
    clean_paths = sorted(glob.glob(f'{clean_dir}/*.wav'))
    separated_paths = sorted(glob.glob(f'{separated_dir}/*.wav'))
    
    # Split train/val
    split_idx = int(len(clean_paths) * (1 - val_split))
    
    # Create datasets
    preprocessor = HighFrequencyArtifactPreprocessor()
    train_dataset = ArtifactDataset(
        clean_paths[:split_idx],
        separated_paths[:split_idx],
        preprocessor
    )
    val_dataset = ArtifactDataset(
        clean_paths[split_idx:],
        separated_paths[split_idx:],
        preprocessor
    )
    
    # Create dataloaders
    train_loader = DataLoader(
        train_dataset,
        batch_size=batch_size,
        shuffle=True,
        num_workers=4
    )
    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        shuffle=False,
        num_workers=4
    )
    
    return train_loader, val_loader

# From separation\base.py
from dataclasses import dataclass
from enum import Enum, auto
from pathlib import Path
from typing import Dict, Optional, Tuple

@dataclass
class SeparationProfile:
    backend: SeparatorBackend
    preserve_high_freq: bool = False
    target_sample_rate: int = 44100
    min_segment_length: float = 5.0
    # Added enhancement parameters
    use_phase_aware_controlnet: bool = False
    use_high_freq_processor: bool = True
    artifact_reduction_config: Optional[ProcessingConfig] = None
    controlnet_model_path: Optional[Path] = None

@dataclass
class SeparationResult:
    clean_vocal: AudioSegment
    separated_vocal: AudioSegment
    enhanced_vocal: Optional[AudioSegment]  # After artifact reduction
    accompaniment: AudioSegment
    mixed: AudioSegment
    analysis_path: Optional[Path] = None
    phase_analysis: Optional[Dict[str, Any]] = None
    artifact_analysis: Optional[Dict[str, str]] = None

class StemProcessor:
    def __init__(self, profile: SeparationProfile):
        self.profile = profile
        self.separator = self._create_separator()
        self.analyzer = VocalSeparationAnalyzer(Path("analysis_output"))
        
        # Initialize enhancement components
        self.artifact_processor = IntegratedHighFrequencyProcessor(
            "enhancement_output",
            config=profile.artifact_reduction_config or ProcessingConfig()
        ) if profile.use_high_freq_processor else None
        
        self.controlnet = PhaseAwareControlNet.from_pretrained(
            profile.controlnet_model_path
        ) if profile.use_phase_aware_controlnet else None

    def process_stems(self, 
                     vocal_paths: Tuple[str, str],
                     accompaniment_paths: Tuple[str, str],
                     start_time: float = 0.0,
                     duration: float = 30.0) -> SeparationResult:
        try:
            self.separator.setup()
            
            # Load and prepare audio
            vocals = self._load_stereo_pair(*vocal_paths, start_time, duration)
            accompaniment = self._load_stereo_pair(*accompaniment_paths, start_time, duration)
            mixed = AudioSegment(
                audio=vocals.audio + accompaniment.audio,
                sample_rate=vocals.sample_rate,
                start_time=start_time,
                duration=duration
            )

            # Initial separation
            separated = self.separator.separate(mixed)
            base_analysis = self.analyzer.analyze(vocals, separated)
            
            # Enhancement pipeline
            enhanced = separated
            artifact_analysis = None
            
            # Apply high-frequency artifact reduction if enabled
            if self.artifact_processor:
                enhanced_audio, analysis_files = self.artifact_processor.process_and_analyze(
                    enhanced.audio
                )
                enhanced = AudioSegment(
                    audio=enhanced_audio,
                    sample_rate=enhanced.sample_rate,
                    start_time=enhanced.start_time,
                    duration=enhanced.duration
                )
                artifact_analysis = analysis_files
            
            # Apply ControlNet enhancement if enabled
            if self.controlnet:
                enhanced = self._apply_controlnet_enhancement(enhanced)
            
            return SeparationResult(
                clean_vocal=vocals,
                separated_vocal=separated,
                enhanced_vocal=enhanced,
                accompaniment=accompaniment,
                mixed=mixed,
                analysis_path=base_analysis,
                artifact_analysis=artifact_analysis
            )
        finally:
            self.separator.cleanup()

    def _apply_controlnet_enhancement(self, audio: AudioSegment) -> AudioSegment:
        # Convert to spectrogram
        spec = self.artifact_processor.create_complex_spectrogram(
            audio.audio
        ) if self.artifact_processor else None
        
        # Process through ControlNet
        enhanced_spec = self.controlnet(spec)
        
        # Convert back to audio
        enhanced_audio = self.artifact_processor.spectrogram_to_audio(
            enhanced_spec
        ) if self.artifact_processor else None
        
        return AudioSegment(
            audio=enhanced_audio,
            sample_rate=audio.sample_rate,
            start_time=audio.start_time,
            duration=audio.duration
        )

# From analysis\base.py
from abc import ABC, abstractmethod
from pathlib import Path
# from ..core.audio import AudioSegment  # Original relative import

class VocalAnalyzer(ABC):
    """Base class for vocal analysis"""
    
    def __init__(self, output_dir: Path):
        self.output_dir = output_dir
        self.output_dir.mkdir(parents=True, exist_ok=True)

    @abstractmethod
    def analyze(self, clean: AudioSegment, separated: AudioSegment) -> Path:
        """Perform analysis and return path to analysis results"""
        pass

    @abstractmethod
    def _create_spectrograms(self, clean: np.ndarray, separated: np.ndarray, 
                          sr: int, output_path: Path):
        """Create and save spectrogram comparisons"""
        pass

# From analysis\artifacts\spectral.py
import librosa
import numpy as np
from pathlib import Path
from typing import Dict, Tuple
# from ...core.audio import AudioSegment  # Original relative import


class SpectralAnalyzer:
    """Spectral analysis and visualization"""
    
    def __init__(self, output_dir: Path, config: Optional[ProcessingConfig] = None):
        self.output_dir = output_dir
        self.output_dir.mkdir(parents=True, exist_ok=True)
        self.config = config or ProcessingConfig()
        self.normalization_params = {}
        
    def _save_comparison(self, 
                        clean_spec: np.ndarray, 
                        separated_spec: np.ndarray, 
                        path: Path):
        """Save visual comparison of spectrograms"""
        fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(12, 16))
        
        # Plot clean spectrogram
        self._plot_spectrogram(
            clean_spec, 
            ax1, 
            "Clean Vocal Spectrogram",
            self.config.sample_rate
        )
        
        # Plot separated spectrogram
        self._plot_spectrogram(
            separated_spec, 
            ax2, 
            "Separated Vocal Spectrogram",
            self.config.sample_rate
        )
        
        # Plot difference
        difference = librosa.amplitude_to_db(
            np.abs(separated_spec) - np.abs(clean_spec),
            ref=np.max
        )
        self._plot_spectrogram(
            difference,
            ax3,
            "Difference (Separated - Clean)",
            self.config.sample_rate,
            cmap='RdBu'
        )
        
        plt.tight_layout()
        plt.savefig(path)
        plt.close()
        
    def _plot_spectrogram(self,
                         spec: np.ndarray,
                         ax: plt.Axes,
                         title: str,
                         sr: int,
                         cmap: str = 'magma'):
        """Helper to plot a single spectrogram"""
        librosa.display.specshow(
            librosa.amplitude_to_db(np.abs(spec), ref=np.max),
            y_axis='mel',
            x_axis='time',
            sr=sr,
            hop_length=self.config.hop_length,
            ax=ax,
            cmap=cmap
        )
        ax.set_title(title, color='white')
        ax.set_xlabel('Time (s)', color='white')
        ax.set_ylabel('Frequency (Hz)', color='white')
        ax.tick_params(colors='white')
        plt.colorbar(ax.collections[0], ax=ax, format='%+2.0f dB')
    
    def _analyze_differences(self, 
                           clean_spec: np.ndarray, 
                           separated_spec: np.ndarray) -> Dict:
        """Analyze spectral differences between clean and separated audio"""
        # Get magnitude spectrograms
        clean_mag = np.abs(clean_spec)
        sep_mag = np.abs(separated_spec)
        
        # Get phase information
        clean_phase = np.angle(clean_spec)
        sep_phase = np.angle(separated_spec)
        
        # Calculate frequency bands
        freq_bins = librosa.fft_frequencies(
            sr=self.config.sample_rate,
            n_fft=self.config.n_fft
        )
        
        bands = {
            "sub_bass": (20, 60),
            "bass": (60, 250),
            "low_mid": (250, 500),
            "mid": (500, 2000),
            "high_mid": (2000, 4000),
            "presence": (4000, 6000),
            "brilliance": (6000, 20000)
        }
        
        analysis = {}
        
        # Analyze each frequency band
        for band_name, (low_freq, high_freq) in bands.items():
            # Get band indices
            band_mask = (freq_bins >= low_freq) & (freq_bins <= high_freq)
            
            # Magnitude difference in band
            mag_diff = np.mean(np.abs(sep_mag[band_mask] - clean_mag[band_mask]))
            
            # Phase coherence in band
            phase_diff = np.abs(sep_phase[band_mask] - clean_phase[band_mask])
            phase_coherence = np.mean(np.cos(phase_diff))
            
            # RMS energy ratio
            clean_rms = np.sqrt(np.mean(clean_mag[band_mask] ** 2))
            sep_rms = np.sqrt(np.mean(sep_mag[band_mask] ** 2))
            energy_ratio = sep_rms / clean_rms if clean_rms > 0 else 0
            
            analysis[band_name] = {
                "magnitude_difference": float(mag_diff),
                "phase_coherence": float(phase_coherence),
                "energy_ratio": float(energy_ratio)
            }
        
        # Overall statistics
        analysis["overall"] = {
            "total_magnitude_difference": float(np.mean(np.abs(sep_mag - clean_mag))),
            "average_phase_coherence": float(np.mean(np.cos(np.abs(sep_phase - clean_phase)))),
            "total_energy_ratio": float(np.sum(sep_mag) / np.sum(clean_mag))
        }
        
        return analysis
    
    def _save_analysis(self, analysis: Dict, path: Path):
        """Save analysis results as JSON"""
        with open(path, 'w') as f:
            json.dump(analysis, f, indent=2)

# From io\audio.py
from pathlib import Path
import numpy as np
import soundfile as sf
import librosa
from typing import Tuple
# from ..core.audio import AudioSegment  # Original relative import

def load_audio_file(path: str, sr: int = 44100, mono: bool = False) -> Tuple[np.ndarray, int]:
    """Load audio file with error handling and validation"""
    try:
        audio, file_sr = librosa.load(path, sr=sr, mono=mono)
        return audio, file_sr
    except Exception as e:
        raise RuntimeError(f"Error loading audio file {path}: {str(e)}")

def save_audio_file(audio: AudioSegment, path: Path) -> None:
    """Save audio file with proper format handling"""
    try:
        # Handle different array shapes
        audio_to_save = audio.audio
        if audio.is_stereo:
            # Convert from (2, samples) to (samples, 2)
            audio_to_save = audio.audio.T
            
        sf.write(str(path), audio_to_save, audio.sample_rate)
    except Exception as e:
        raise RuntimeError(f"Error saving audio file {path}: {str(e)}")

# From enhancement\base.py
from abc import ABC, abstractmethod
from typing import Optional
# from ...core.audio import AudioSegment  # Original relative import
# from ...core.types import ProcessingConfig  # Original relative import

class EnhancementProcessor(ABC):
    """Base class for audio enhancement processors"""
    
    def __init__(self, config: Optional[ProcessingConfig] = None):
        self.config = config or ProcessingConfig()
    
    @abstractmethod
    def enhance(self, audio: AudioSegment) -> AudioSegment:
        """Enhance audio segment"""
        pass
    
    @abstractmethod
    def validate(self, audio: AudioSegment) -> dict:
        """Validate enhancement results"""
        pass

# From preparation\base.py
from abc import ABC, abstractmethod
from pathlib import Path
from typing import Tuple, Dict
# from ..core.types import SeparationResult  # Original relative import
# from ..core.audio import AudioSegment  # Original relative import

class VocalSeparator(ABC):
    """Abstract base class for vocal separators"""
    
    def __init__(self, output_dir: str = "output"):
        self.output_dir = Path(output_dir)
        self.output_dir.mkdir(parents=True, exist_ok=True)

    @abstractmethod
    def separate_and_analyze(self,
                           vocal_paths: Tuple[str, str],
                           accompaniment_paths: Tuple[str, str],
                           start_time: float = 0.0,
                           duration: float = 30.0,
                           run_analysis: bool = True) -> SeparationResult:
        """Perform separation and analysis"""
        pass

    @abstractmethod
    def _load_stereo_pair(self, left_path: str, right_path: str, 
                         start_time: float, duration: float) -> AudioSegment:
        """Load and process stereo pair"""
        pass

    @abstractmethod
    def _separate_vocals(self, mixed: AudioSegment) -> AudioSegment:
        """Perform vocal separation"""
        pass

    @abstractmethod
    def _save_audio_files(self, vocals: AudioSegment, 
                         accompaniment: AudioSegment,
                         mixed: AudioSegment, 
                         separated: AudioSegment,
                         start_time: float) -> Dict[str, Path]:
        """Save all audio files"""
        pass

    def cleanup(self):
        """Cleanup resources - override if needed"""
        pass

    def __enter__(self):
        return self

    def __exit__(self, exc_type, exc_val, exc_tb):
        self.cleanup()

# From separation\spleeter.py
import tensorflow as tf
from pathlib import Path
import numpy as np
from typing import Dict, Tuple, Optional
from datetime import datetime
from spleeter.separator import Separator as SpleeterBase

# from .base import VocalSeparator  # Original relative import
# from ..core.audio import AudioSegment  # Original relative import
# from ..core.types import SeparationResult  # Original relative import
# from ..io.audio import load_audio_file, save_audio_file  # Original relative import
# from ..analysis.spectral import SpectralAnalyzer  # Original relative import

class SpleeterSeparator(VocalSeparator):
    """Concrete implementation using Spleeter"""
    
    def __init__(self, output_dir: str = "output"):
        super().__init__(output_dir)
        
        # Initialize components
        self.analyzer = SpectralAnalyzer(self.output_dir)
        
        # Defer TensorFlow setup until needed
        self.separator = None
        self.graph = None
        self.session = None

    def _setup_tensorflow(self):
        """Setup TensorFlow session and graph - called only when needed"""
        if self.separator is not None:
            return  # Already initialized
            
        # Create a new graph and session without resetting
        self.graph = tf.Graph()
        self.graph.as_default().__enter__()
        
        config = tf.compat.v1.ConfigProto()
        config.gpu_options.allow_growth = True
        self.session = tf.compat.v1.Session(graph=self.graph, config=config)
        self.session.as_default().__enter__()
        
        # Initialize Spleeter only after graph/session setup
        self.separator = SpleeterBase('spleeter:2stems')

    def separate_and_analyze(self,
                           vocal_paths: Tuple[str, str],
                           accompaniment_paths: Tuple[str, str],
                           start_time: float = 0.0,
                           duration: float = 30.0,
                           run_analysis: bool = True) -> SeparationResult:
        """Main method to perform separation and analysis"""
        # Ensure TensorFlow is set up
        self._setup_tensorflow()
        
        # Load audio
        vocals = self._load_stereo_pair(*vocal_paths, start_time, duration)
        accompaniment = self._load_stereo_pair(*accompaniment_paths, start_time, duration)

        # Create mix
        mixed = AudioSegment(
            audio=vocals.audio + accompaniment.audio,
            sample_rate=vocals.sample_rate,
            start_time=start_time,
            duration=duration
        )

        # Perform separation
        separated = self._separate_vocals(mixed)
        
        # Save files
        file_paths = self._save_audio_files(
            vocals, accompaniment, mixed, separated, start_time
        )

        result = SeparationResult(
            clean_vocal=vocals,
            separated_vocal=separated,
            accompaniment=accompaniment,
            mixed=mixed,
            file_paths=file_paths
        )

        # Run analysis if requested
        if run_analysis:
            result.analysis_path = self.analyzer.analyze(vocals, separated)

        return result

    def _load_stereo_pair(self, left_path: str, right_path: str, 
                         start_time: float, duration: float) -> AudioSegment:
        """Load and process stereo pair"""
        print(f"Loading {left_path}...")
        left, sr = load_audio_file(left_path, sr=44100, mono=True)
        print(f"Left channel length: {len(left)} samples ({len(left)/44100:.2f} seconds)")

        print(f"Loading {right_path}...")
        right, _ = load_audio_file(right_path, sr=44100, mono=True)
        print(f"Right channel length: {len(right)} samples ({len(right)/44100:.2f} seconds)")

        # Ensure same length
        min_length = min(len(left), len(right))
        left = left[:min_length]
        right = right[:min_length]
        print(f"Adjusted stereo length: {min_length} samples ({min_length/44100:.2f} seconds)")

        # Extract segment
        start_sample = int(start_time * 44100)
        duration_samples = int(duration * 44100)
        
        if start_sample + duration_samples > min_length:
            print(f"Warning: Requested duration extends beyond audio length. Truncating.")
            duration_samples = min_length - start_sample
        
        left_segment = left[start_sample:start_sample + duration_samples]
        right_segment = right[start_sample:start_sample + duration_samples]

        # Stack to stereo
        stereo = np.vstack([left_segment, right_segment])
        
        return AudioSegment(
            audio=stereo,
            sample_rate=44100,
            start_time=start_time,
            duration=duration_samples/44100
        )

    def _separate_vocals(self, mixed: AudioSegment) -> AudioSegment:
        """Perform vocal separation"""
        # Convert to mono and reshape for Spleeter
        mix_mono = mixed.to_mono().audio
        mix_mono = mix_mono.reshape(-1, 1)

        print(f"Mix shape before separation: {mix_mono.shape}")
        print("Running separation...")
        
        separated = self.separator.separate(mix_mono)
        separated_vocals = separated['vocals']
        print(f"Separated vocals shape: {separated_vocals.shape}")
        
        # Since Spleeter returns (samples, channels), we should handle it accordingly
        if len(separated_vocals.shape) == 2:
            if separated_vocals.shape[1] == 2:
                # If it's stereo, convert to our preferred format (2, samples)
                separated_vocals = separated_vocals.T
            elif separated_vocals.shape[1] == 1:
                # If it's mono in (samples, 1) shape, convert to 1D array
                separated_vocals = separated_vocals.reshape(-1)
                
        print(f"Final separated vocals shape: {separated_vocals.shape}")
        
        if len(separated_vocals.shape) == 1:
            # If mono, duplicate to stereo to match input
            separated_vocals = np.vstack([separated_vocals, separated_vocals])
        
        print(f"Output separated vocals shape: {separated_vocals.shape}")
            
        if separated_vocals.shape[1] < 1000:  # Sanity check on the correct dimension
            raise ValueError(f"Separated vocals seem too short: {separated_vocals.shape}")
                
        return AudioSegment(
            audio=separated_vocals,
            sample_rate=mixed.sample_rate,
            start_time=mixed.start_time,
            duration=mixed.duration
        )

    def _save_audio_files(self, vocals: AudioSegment, accompaniment: AudioSegment,
                         mixed: AudioSegment, separated: AudioSegment,
                         start_time: float) -> Dict[str, Path]:
        """Save all audio files"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        save_dir = self.output_dir / f"separation_{timestamp}"
        save_dir.mkdir(exist_ok=True)

        files = {}
        
        # Prepare audio segments for saving
        segments = {
            'clean_vocal': vocals,
            'accompaniment': accompaniment,
            'mix': mixed,
            'separated_vocal': separated
        }

        for name, segment in segments.items():
            path = save_dir / f"{name}_from_{start_time:.1f}s.wav"
            print(f"\nProcessing {name}:")
            print(f"Original shape: {segment.audio.shape}")
            
            save_audio_file(segment, path)
            files[name] = path

        return files

    def cleanup(self):
        """Cleanup resources"""
        try:
            if hasattr(self, 'separator') and hasattr(self.separator, '_get_model'):
                self.separator._get_model.cache_clear()
            
            if hasattr(self, 'session'):
                self.session.as_default().__exit__(None, None, None)
                self.session.close()
                delattr(self, 'session')
            
            if hasattr(self, 'graph'):
                self.graph.as_default().__exit__(None, None, None)
                delattr(self, 'graph')
            
        except Exception as e:
            print(f"Warning during cleanup: {str(e)}")

    @property
    def capabilities(self) -> Dict[str, any]:
        """Report capabilities/limitations of this backend"""
        return {
            "max_frequency": 11000,  # Hz
            "supports_stereo": True,
            "native_sample_rate": 22050,
            "recommended_min_segment": 5.0  # seconds
        }

```
 /\___/\   ChimeraCat
( o   o )  Modular Python Fusion
(  =^=  )  https://github.com/scottvr/chimeracat
 (______)  Generated: 2024-11-20 04:29:43
```
